# Load data

In [ ]:
import glob
import os

In [ ]:
data = []
for n in glob.glob("un/TXT/Session 77 - 2022/*.txt"):
    data.append({"country": os.path.basename(n.replace("_77_2022.txt", "")), "text": open(n).read() })

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', 500)
df = pd.DataFrame(data)
df

# Sentence segmentation

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# runs 30 seconds or so
sentences = []
for text in tqdm(df["text"]):
    doc = nlp(text)
    for sentence in doc.sents:
        sentences.append(str(sentence).strip())

In [ ]:
sentences

In [ ]:
len(sentences)

In [ ]:
open("sentences.txt", "w").write("@@@".join(sentences))

# Encode sentences

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
# can take a minute or two depending on CPU/GPU configuration
sembeddings = model.encode(sentences, show_progress_bar=True)

In [ ]:
import numpy as np
with open("sentences.npy", "wb") as f:
    np.save(f, sembeddings)

In [ ]:
sembeddings.shape

In [ ]:
model2 = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

In [ ]:
sembeddings2 = model2.encode(sentences, show_progress_bar=True)

In [ ]:
with open("sentences2.npy", "wb") as f:
    np.save(f, sembeddings2)

# Retrieval

In [ ]:
def search(query, text, corpus_embeddings, model, top=20):
    # code query to restrict search space
    question_embedding = model.encode(query)
    
    # Determine similarity (vectors are normalized)
    sim = np.dot(corpus_embeddings, question_embedding)
    
    # Get most similar top by sorting
    hits = [ { "text": text[i], "score": sim[i] } 
                     for i in sim.argsort()[::-1][0:top] ]
    
    # Return as dataframe
    return pd.DataFrame(hits)

In [ ]:
pd.set_option('display.max_colwidth', 0)

In [ ]:
search("The climate crisis is worse in poorer countries", sentences, sembeddings, model)

In [ ]:
search("The climate crisis is worse in poorer countries", sentences, sembeddings2, model2)

In [ ]:
search("Die Klimakrise betrifft vor allem ärmere Länder", sentences, sembeddings2, model2)